In [ ]:
import ipywidgets
import IPython.display

prompt = ipywidgets.widgets.Textarea(
    value="White Dog",
    description="Prompt: ",
)

model = ipywidgets.widgets.Dropdown(
    options=["gpt-image-1.5", "gpt-image-1"],
    value="gpt-image-1.5",
    description="Model: ",
)

n = ipywidgets.widgets.IntSlider(
    min=1,
    max=10,
    step=1,
    value=1,
    description="Number: ",
)

quality = ipywidgets.widgets.Dropdown(
    options=["high", "medium", "low"],
    value="high",
    description="Quality: ",
)

size = ipywidgets.widgets.Dropdown(
    options=["1024x1024", "1536x1024", "1024x1536"],
    value="1024x1024",
    description="Size: ",
)

background = ipywidgets.widgets.Dropdown(
    options=["transparent", "opaque", "auto"],
    value="auto",
    description="Style: ",
)

output = ipywidgets.widgets.Output()

button = ipywidgets.widgets.Button(
    description="Generate",
)


@output.capture()
def generate(b: ipywidgets.widgets.Button):
    import requests
    import base64

    b.disabled = True

    print("Generating...")

    r = requests.post("http://cortex-api.cortex-api.svc.cluster.local:8080/v1/images/generations", json={
        "prompt": prompt.value,
        "model": model.value,
        "n": n.value,
        "quality": quality.value,
        "size": size.value,
        "background": background.value,
    }, headers={
        "Content-Type": "application/json"
    })

    response = r.json()

    if "error" in response:
        import sys
        print(response["error"]["message"], file=sys.stderr)
        sys.exit(1)

    images = []
    for image in response.get("data", []):
        data = base64.b64decode(image["b64_json"])
        images.append(IPython.display.Image(data=data))

    IPython.display.display(*images)

    b.disabled = False


button.on_click(generate)

IPython.display.display(prompt, model, n, quality, size, background, button, output)